# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [26]:

#chrome -> chromedriver

In [27]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

#Passo 1 - Abrir o site e pegar a cotação do Dolar
#abrir o navegador
navegador = webdriver.Chrome()
#entrar no google
navegador.get('https://www.google.com.br/')
#pesquisar no google "cotação dolar"
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dolar')
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
#pegar cotação que ta no goole
cotacao_dolar = navegador.find_element('xpath','//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)

#Passo 2 - Pegar a cotação do Euro
navegador.get('https://www.google.com.br/')
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro = navegador.find_element('xpath','//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)

#Passo 3 - Pegar a cotação do ouro
navegador.get('https://www.melhorcambio.com/ouro-hoje')
cotacao_ouro = navegador.find_element('xpath','//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',','.')
print(cotacao_ouro)

navegador.quit()


5.2048499999999995
5.06848293
284.72


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [28]:
import pandas as pd

#Passo 4 - importar a base de preços
tabela = pd.read_excel('Produtos.xlsx')
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [29]:
# Passo 5: Recalcular o preço de cada produto
# atualizar a cotação
# nas linhas onde na coluna "Moeda" = Dólar
#tabela.loc['Linha', 'Coluna'](Loc = Localizar)
tabela.loc[tabela['Moeda']== 'Dólar', 'Cotação'] = float(cotacao_dolar)
tabela.loc[tabela['Moeda']== 'Euro', 'Cotação'] = float(cotacao_euro)
tabela.loc[tabela['Moeda']== 'Ouro', 'Cotação'] = float(cotacao_ouro)

# atualizar o preço de compras (preço original * cotação)
tabela['Preço de Compra'] = tabela['Preço Original'] * tabela['Cotação']

# atualizar o preço de venda (preço de compra * Margem)
tabela['Preço de Venda'] = tabela['Preço de Compra'] * tabela['Margem']

tabela["Preço de Venda"] = tabela["Preço de Venda"].map("R${:.2f}".format)

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.204850,5204.797951,1.40,R$7286.72
1,Carro Renault,4500.00,Euro,5.068483,22808.173185,2.00,R$45616.35
2,Notebook Dell,899.99,Dólar,5.204850,4684.312952,1.70,R$7963.33
3,IPhone,799.00,Dólar,5.204850,4158.675150,1.70,R$7069.75
4,Carro Fiat,3000.00,Euro,5.068483,15205.448790,1.90,R$28890.35
5,Celular Xiaomi,480.48,Dólar,5.204850,2500.826328,2.00,R$5001.65
6,Joia 20g,20.00,Ouro,284.720000,5694.400000,1.15,R$6548.56


### Agora vamos exportar a nova base de preços atualizada

In [30]:
#Passo 5 - Exportar base de preços para o excel
tabela.to_excel('Produtos Novos.xlsx', index = False)